In [1]:
import pandas as pd
import requests

# Enlace de la API con los datos
api_url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# Realizar la solicitud HTTP para obtener los datos
try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza una excepción para errores HTTP

    # Cargar los datos JSON
    data = response.json()

    # Convertir los datos a un DataFrame de Pandas
    df = pd.DataFrame(data)

    print("¡Datos cargados exitosamente desde la API y convertidos a DataFrame!")
    print("\nPrimeras 5 filas del DataFrame:")
    print(df.head())
    print(f"\nDimensiones del DataFrame: {df.shape}")

except requests.exceptions.RequestException as e:
    print(f"Error al cargar los datos desde la API: {e}")
except ValueError as e:
    print(f"Error al procesar los datos JSON: {e}")

¡Datos cargados exitosamente desde la API y convertidos a DataFrame!

Primeras 5 filas del DataFrame:
   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'Male', 'SeniorCitizen': 1, 'Partne...   
4  0013-EXCHZ   Yes  {'gender': 'Female', 'SeniorCitizen': 1, 'Part...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
3   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
4   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetSer

In [3]:
import pandas as pd
import requests
# La importación de json_normalize cambió. Ahora se accede directamente desde pandas.
# from pandas.io.json import json_normalize # Comentado porque ya no es necesario
from pandas import json_normalize # Importación corregida

# Enlace de la API con los datos
api_url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# Realizar la solicitud HTTP para obtener los datos
try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza una excepción para errores HTTP

    # Cargar los datos JSON
    data = response.json()

    # Normalizar las columnas anidadas
    df_customer = json_normalize(data, sep='_').filter(regex='^customer_')
    df_customer.columns = df_customer.columns.str.replace('customer_', '')

    df_phone = json_normalize(data, sep='_').filter(regex='^phone_')
    df_phone.columns = df_phone.columns.str.replace('phone_', '')

    df_internet = json_normalize(data, sep='_').filter(regex='^internet_')
    df_internet.columns = df_internet.columns.str.replace('internet_', '')

    df_account = json_normalize(data, sep='_').filter(regex='^account_')
    df_account.columns = df_account.columns.str.replace('account_', '')

    df_base = pd.DataFrame(data)[['customerID', 'Churn']]
    df = pd.concat([df_base, df_customer, df_phone, df_internet, df_account], axis=1)

    print("¡Datos cargados y normalizados para la comprobación y manejo de inconsistencias!")
    print(f"\nDimensiones del DataFrame: {df.shape}")
    print("\nInformación del DataFrame (columnas y tipos de datos antes del manejo de inconsistencias):")
    df.info()

    # --- Manejo de Inconsistencias ---

    print("\n--- Manejo de Inconsistencias en 'TotalCharges' ---")
    # Convertir 'TotalCharges' a numérico. Los valores vacíos o no numéricos se convertirán a NaN.
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

    # Reemplazar NaN en 'TotalCharges' por 0.0 (asumiendo que clientes sin cargo total inicial tienen 0)
    # Es una suposición común para clientes nuevos que aún no han sido facturados.
    df['TotalCharges'] = df['TotalCharges'].fillna(0.0)

    # Verificar si quedan valores no numéricos en 'TotalCharges'
    non_numeric_after_handling = df[pd.to_numeric(df['TotalCharges'], errors='coerce').isnull()].shape[0]
    if non_numeric_after_handling == 0:
        print("La columna 'TotalCharges' ha sido limpiada y convertida a tipo numérico (float64).")
    else:
        print(f"Advertencia: Aún quedan {non_numeric_after_handling} valores no numéricos en 'TotalCharges' después del manejo.")


    print("\n--- Manejo de Inconsistencias Generales (ej. capitalización, espacios) ---")
    # Iterar sobre las columnas de tipo 'object' (strings)
    for col in df.select_dtypes(include='object').columns:
        # Excluir 'customerID' ya que es un identificador único y no necesita normalización de texto
        # Excluir 'Churn' por ahora, ya que la transformaremos a binario más adelante
        if col not in ['customerID', 'Churn', 'TotalCharges']: # Excluir TotalCharges que ya se trató
            # Convertir a minúsculas y eliminar espacios en blanco al inicio/final
            df[col] = df[col].astype(str).str.strip().str.lower()
            print(f"Columna '{col}': Convertida a minúsculas y espacios eliminados.")
            # Opcional: reemplazar variaciones comunes
            df[col] = df[col].replace({'no internet service': 'no_internet_service',
                                       'no phone service': 'no_phone_service'})


    # Convertir 'SeniorCitizen' a tipo 'object' si aún no lo está, ya que es una categoría binaria (0/1)
    # y puede ser más claro tratarla como categórica si no se va a operar matemáticamente.
    # Si se va a usar directamente en un modelo, int64 está bien. Aquí, para EDA, la dejamos como int64.
    # df['SeniorCitizen'] = df['SeniorCitizen'].astype(object)

    print("\n--- Verificación de Tipos de Datos después del Manejo de Inconsistencias ---")
    print(df.dtypes)
    print("\nInformación del DataFrame (columnas y tipos de datos después del manejo de inconsistencias):")
    df.info()

except requests.exceptions.RequestException as e:
    print(f"Error al cargar los datos desde la API: {e}")
except ValueError as e:
    print(f"Error al procesar los datos JSON: {e}")

¡Datos cargados y normalizados para la comprobación y manejo de inconsistencias!

Dimensiones del DataFrame: (7267, 21)

Información del DataFrame (columnas y tipos de datos antes del manejo de inconsistencias):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  

KeyError: 'TotalCharges'